<a href="https://colab.research.google.com/github/sagasucksatlife1/COMMISION_REPO/blob/main/BACKEND_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q fastapi uvicorn pydantic nest-asyncio pyngrok

import nest_asyncio
from pyngrok import ngrok
import uvicorn
from threading import Thread
import time
import sqlite3
from datetime import datetime
import hashlib
import secrets
from fastapi import FastAPI, HTTPException, Depends, Header
from fastapi.middleware.cors import CORSMiddleware
from fastapi.responses import StreamingResponse
from pydantic import BaseModel
from typing import Optional
import io
import csv

nest_asyncio.apply()

# Database class with /tmp/ path
class Database:
    def __init__(self, db_name="/tmp/commission_system.db"):  # FIXED PATH
        self.db_name = db_name
        self.init_db()

    def get_connection(self):
        conn = sqlite3.connect(self.db_name)
        conn.row_factory = sqlite3.Row
        return conn

    def init_db(self):
        conn = self.get_connection()
        cursor = conn.cursor()
        cursor.execute('''CREATE TABLE IF NOT EXISTS users (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            username TEXT UNIQUE NOT NULL,
            password_hash TEXT NOT NULL,
            role TEXT NOT NULL CHECK(role IN ('employee', 'admin')),
            full_name TEXT NOT NULL,
            created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
        )''')
        cursor.execute('''CREATE TABLE IF NOT EXISTS commissions (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            user_id INTEGER NOT NULL,
            sale_date DATE NOT NULL,
            unlisted_sales REAL NOT NULL DEFAULT 0,
            loans REAL NOT NULL DEFAULT 0,
            third_party_sales REAL NOT NULL DEFAULT 0,
            calculated_commission REAL NOT NULL,
            status TEXT NOT NULL DEFAULT 'pending' CHECK(status IN ('pending', 'approved', 'rejected')),
            created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
            updated_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
            FOREIGN KEY (user_id) REFERENCES users(id)
        )''')
        cursor.execute('''CREATE TABLE IF NOT EXISTS sessions (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            user_id INTEGER NOT NULL,
            token TEXT UNIQUE NOT NULL,
            created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
            expires_at TIMESTAMP NOT NULL,
            FOREIGN KEY (user_id) REFERENCES users(id)
        )''')
        conn.commit()
        conn.close()
        self.create_default_admin()

    def hash_password(self, password):
        return hashlib.sha256(password.encode()).hexdigest()

    def create_default_admin(self):
        conn = self.get_connection()
        cursor = conn.cursor()
        cursor.execute("SELECT * FROM users WHERE username = ?", ("admin",))
        if not cursor.fetchone():
            password_hash = self.hash_password("admin123")
            cursor.execute('INSERT INTO users (username, password_hash, role, full_name) VALUES (?, ?, ?, ?)',
                         ("admin", password_hash, "admin", "System Admin"))
            conn.commit()
        conn.close()

    def create_user(self, username, password, role, full_name):
        conn = self.get_connection()
        cursor = conn.cursor()
        try:
            cursor.execute('INSERT INTO users (username, password_hash, role, full_name) VALUES (?, ?, ?, ?)',
                         (username, self.hash_password(password), role, full_name))
            conn.commit()
            user_id = cursor.lastrowid
            conn.close()
            return user_id
        except sqlite3.IntegrityError:
            conn.close()
            return None

    def verify_user(self, username, password):
        conn = self.get_connection()
        cursor = conn.cursor()
        cursor.execute('SELECT id, username, role, full_name FROM users WHERE username = ? AND password_hash = ?',
                      (username, self.hash_password(password)))
        user = cursor.fetchone()
        conn.close()
        return dict(user) if user else None

    def create_session(self, user_id):
        conn = self.get_connection()
        cursor = conn.cursor()
        token = secrets.token_urlsafe(32)
        expires_at = datetime.now().timestamp() + (24 * 60 * 60)
        cursor.execute('INSERT INTO sessions (user_id, token, expires_at) VALUES (?, ?, datetime(?, "unixepoch"))',
                      (user_id, token, expires_at))
        conn.commit()
        conn.close()
        return token

    def verify_session(self, token):
        conn = self.get_connection()
        cursor = conn.cursor()
        cursor.execute('''SELECT s.user_id, u.username, u.role, u.full_name FROM sessions s
                         JOIN users u ON s.user_id = u.id WHERE s.token = ? AND s.expires_at > datetime('now')''', (token,))
        session = cursor.fetchone()
        conn.close()
        return dict(session) if session else None

    def delete_session(self, token):
        conn = self.get_connection()
        cursor = conn.cursor()
        cursor.execute("DELETE FROM sessions WHERE token = ?", (token,))
        conn.commit()
        conn.close()

    def create_commission(self, user_id, sale_date, unlisted_sales, loans, third_party_sales):
        calculated_commission = (loans / 3) + (unlisted_sales / 3) + (third_party_sales * 100)
        conn = self.get_connection()
        cursor = conn.cursor()
        cursor.execute('INSERT INTO commissions (user_id, sale_date, unlisted_sales, loans, third_party_sales, calculated_commission) VALUES (?, ?, ?, ?, ?, ?)',
                      (user_id, sale_date, unlisted_sales, loans, third_party_sales, calculated_commission))
        conn.commit()
        commission_id = cursor.lastrowid
        conn.close()
        return commission_id

    def update_commission(self, commission_id, user_id, sale_date, unlisted_sales, loans, third_party_sales):
        calculated_commission = (loans / 3) + (unlisted_sales / 3) + (third_party_sales * 100)
        conn = self.get_connection()
        cursor = conn.cursor()
        cursor.execute('UPDATE commissions SET sale_date=?, unlisted_sales=?, loans=?, third_party_sales=?, calculated_commission=?, updated_at=CURRENT_TIMESTAMP WHERE id=? AND user_id=?',
                      (sale_date, unlisted_sales, loans, third_party_sales, calculated_commission, commission_id, user_id))
        conn.commit()
        affected = cursor.rowcount
        conn.close()
        return affected > 0

    def delete_commission(self, commission_id, user_id):
        conn = self.get_connection()
        cursor = conn.cursor()
        cursor.execute('DELETE FROM commissions WHERE id=? AND user_id=?', (commission_id, user_id))
        conn.commit()
        affected = cursor.rowcount
        conn.close()
        return affected > 0

    def get_user_commissions(self, user_id, months=1):
        conn = self.get_connection()
        cursor = conn.cursor()
        cursor.execute('SELECT * FROM commissions WHERE user_id=? AND sale_date >= date("now", "-" || ? || " months") ORDER BY sale_date DESC',
                      (user_id, months))
        commissions = [dict(row) for row in cursor.fetchall()]
        conn.close()
        return commissions

    def get_all_commissions(self, months=1, employee_id=None):
        conn = self.get_connection()
        cursor = conn.cursor()
        if employee_id:
            cursor.execute('SELECT c.*, u.full_name as employee_name FROM commissions c JOIN users u ON c.user_id=u.id WHERE c.user_id=? AND c.sale_date >= date("now", "-" || ? || " months") ORDER BY c.sale_date DESC',
                          (employee_id, months))
        else:
            cursor.execute('SELECT c.*, u.full_name as employee_name FROM commissions c JOIN users u ON c.user_id=u.id WHERE c.sale_date >= date("now", "-" || ? || " months") ORDER BY c.sale_date DESC',
                          (months,))
        commissions = [dict(row) for row in cursor.fetchall()]
        conn.close()
        return commissions

    def update_commission_status(self, commission_id, status):
        conn = self.get_connection()
        cursor = conn.cursor()
        cursor.execute('UPDATE commissions SET status=?, updated_at=CURRENT_TIMESTAMP WHERE id=?', (status, commission_id))
        conn.commit()
        affected = cursor.rowcount
        conn.close()
        return affected > 0

    def get_all_employees(self):
        conn = self.get_connection()
        cursor = conn.cursor()
        cursor.execute('SELECT id, username, full_name FROM users WHERE role="employee" ORDER BY full_name')
        employees = [dict(row) for row in cursor.fetchall()]
        conn.close()
        return employees

# FastAPI app
app = FastAPI()
app.add_middleware(CORSMiddleware, allow_origins=["*"], allow_credentials=True, allow_methods=["*"], allow_headers=["*"])
db = Database()

class LoginRequest(BaseModel):
    username: str
    password: str

class CreateUserRequest(BaseModel):
    username: str
    password: str
    role: str
    full_name: str

class CommissionRequest(BaseModel):
    sale_date: str
    unlisted_sales: float
    loans: float
    third_party_sales: float

class UpdateCommissionRequest(BaseModel):
    commission_id: int
    sale_date: str
    unlisted_sales: float
    loans: float
    third_party_sales: float

class UpdateStatusRequest(BaseModel):
    commission_id: int
    status: str

def get_current_user(authorization: Optional[str] = Header(None)):
    if not authorization:
        raise HTTPException(status_code=401, detail="Not authenticated")
    user = db.verify_session(authorization.replace("Bearer ", ""))
    if not user:
        raise HTTPException(status_code=401, detail="Invalid token")
    return user

def require_admin(current_user: dict = Depends(get_current_user)):
    if current_user["role"] != "admin":
        raise HTTPException(status_code=403, detail="Admin access required")
    return current_user

@app.post("/api/auth/login")
def login(request: LoginRequest):
    user = db.verify_user(request.username, request.password)
    if not user:
        raise HTTPException(status_code=401, detail="Invalid credentials")
    token = db.create_session(user["id"])
    return {"token": token, "user": user}

@app.post("/api/auth/logout")
def logout(authorization: Optional[str] = Header(None)):
    if authorization:
        db.delete_session(authorization.replace("Bearer ", ""))
    return {"message": "Logged out"}

@app.get("/api/auth/me")
def get_me(current_user: dict = Depends(get_current_user)):
    return current_user

@app.post("/api/users/create")
def create_user(request: CreateUserRequest, _: dict = Depends(require_admin)):
    user_id = db.create_user(request.username, request.password, request.role, request.full_name)
    if not user_id:
        raise HTTPException(status_code=400, detail="Username exists")
    return {"message": "User created", "user_id": user_id}

@app.get("/api/users/employees")
def get_employees(_: dict = Depends(require_admin)):
    return {"employees": db.get_all_employees()}

@app.post("/api/commissions/create")
def create_commission(request: CommissionRequest, current_user: dict = Depends(get_current_user)):
    commission_id = db.create_commission(current_user["user_id"], request.sale_date, request.unlisted_sales, request.loans, request.third_party_sales)
    return {"message": "Created", "commission_id": commission_id}

@app.put("/api/commissions/update")
def update_commission(request: UpdateCommissionRequest, current_user: dict = Depends(get_current_user)):
    if not db.update_commission(request.commission_id, current_user["user_id"], request.sale_date, request.unlisted_sales, request.loans, request.third_party_sales):
        raise HTTPException(status_code=404, detail="Not found")
    return {"message": "Updated"}

@app.delete("/api/commissions/{commission_id}")
def delete_commission(commission_id: int, current_user: dict = Depends(get_current_user)):
    if not db.delete_commission(commission_id, current_user["user_id"]):
        raise HTTPException(status_code=404, detail="Not found")
    return {"message": "Deleted"}

@app.get("/api/commissions/my")
def get_my_commissions(months: int = 1, current_user: dict = Depends(get_current_user)):
    return {"commissions": db.get_user_commissions(current_user["user_id"], months)}

@app.get("/api/commissions/all")
def get_all_commissions(months: int = 1, employee_id: Optional[int] = None, _: dict = Depends(require_admin)):
    return {"commissions": db.get_all_commissions(months, employee_id)}

@app.put("/api/commissions/status")
def update_status(request: UpdateStatusRequest, _: dict = Depends(require_admin)):
    if not db.update_commission_status(request.commission_id, request.status):
        raise HTTPException(status_code=404, detail="Not found")
    return {"message": "Updated"}

@app.get("/api/commissions/export")
def export_commissions(months: int = 1, employee_id: Optional[int] = None, _: dict = Depends(require_admin)):
    commissions = db.get_all_commissions(months, employee_id)
    output = io.StringIO()
    writer = csv.writer(output)
    writer.writerow(["Employee", "Date", "Unlisted", "Loans", "Third-Party", "Commission", "Status"])
    for c in commissions:
        writer.writerow([c["employee_name"], c["sale_date"], c["unlisted_sales"], c["loans"], c["third_party_sales"], c["calculated_commission"], c["status"]])
    output.seek(0)
    return StreamingResponse(io.BytesIO(output.getvalue().encode()), media_type="text/csv", headers={"Content-Disposition": f"attachment; filename=commissions_{datetime.now().strftime('%Y%m%d')}.csv"})

@app.get("/")
def root():
    return {"message": "Commission System API", "status": "running"}

# Start server
ngrok.set_auth_token("37YbDrJGa6liIuCfJLbZENBcn26_4nca2ggo2G1ULHserGqcV")
public_url = ngrok.connect(8000).public_url

print("=" * 70)
print(f"🚀 SERVER: {public_url}")
print(f"📋 DOCS: {public_url}/docs")
print("=" * 70)

def run_server():
    uvicorn.run(app, host="0.0.0.0", port=8000, log_level="error")

Thread(target=run_server, daemon=True).start()

print("✅ Server running. Keep this cell alive.\n")

while True:
    time.sleep(1)

Exception in thread Thread-4 (run_server):
Traceback (most recent call last):
  File "/usr/lib/python3.12/threading.py", line 1075, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.12/threading.py", line 1012, in run
    self._target(*self._args, **self._kwargs)
  File "/tmp/ipython-input-355819299.py", line 333, in run_server
  File "/usr/local/lib/python3.12/dist-packages/uvicorn/main.py", line 579, in run
    server.run()
  File "/usr/local/lib/python3.12/dist-packages/uvicorn/server.py", line 65, in run
    return asyncio.run(self.serve(sockets=sockets))
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/nest_asyncio.py", line 26, in run
    loop = asyncio.get_event_loop()
           ^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/nest_asyncio.py", line 40, in _get_event_loop
    loop = events.get_event_loop_policy().get_event_loop()
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File

🚀 SERVER: https://overcultured-uretic-codi.ngrok-free.dev
📋 DOCS: https://overcultured-uretic-codi.ngrok-free.dev/docs
✅ Server running. Keep this cell alive.



/usr/lib/python3.12/threading.py:1077: RuntimeWarning: coroutine 'Server.serve' was never awaited
  self._invoke_excepthook(self)
